In [1]:
from urllib.request import urlretrieve
import tensorflow as tf
import os
from tensorflow import keras
from keras.layers import Dense , Flatten , MaxPooling2D , Dropout ,BatchNormalization
from keras import Model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import RMSprop

In [2]:
!pip install -q kaggle

In [10]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"praneethratna","key":"4997eb4fb4e67eb4ec38a0e771ad035f"}'}

In [16]:
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [17]:
!kaggle datasets download -d andrewmvd/animal-faces

 99% 688M/696M [00:07<00:00, 120MB/s]
100% 696M/696M [00:07<00:00, 94.4MB/s]


In [ ]:
!mkdir train
!unzip animal-faces.zip

In [19]:
weightsurl = "https://github.com/fchollet/deep-learning-models/releases/download/v0.5/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weightspath = 'inceptionweights.h5'
urlretrieve(weightsurl,weightspath)

('inceptionweights.h5', <http.client.HTTPMessage at 0x7f9d185e3e90>)

In [20]:
TRAIN_DIR = '/content/' + '/afhq/train/'
VAL_DIR = '/content/' + '/afhq/val/'
train_datagen = ImageDataGenerator(rescale = 1./255,width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True,vertical_flip=False)
train = train_datagen.flow_from_directory(TRAIN_DIR,target_size = (256,256),batch_size = 256,class_mode = 'categorical')
val_datagen = ImageDataGenerator(rescale = 1./255,width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True,vertical_flip=False)
val = val_datagen.flow_from_directory(VAL_DIR,target_size = (256,256),batch_size = 256,class_mode = 'categorical')

Found 14630 images belonging to 3 classes.
Found 1500 images belonging to 3 classes.


In [21]:
pre_model = InceptionV3(input_shape = (256,256,3),include_top = False,weights = None)
pre_model.load_weights('inceptionweights.h5')

for layer in pre_model.layers:
  layer.trainable = False

#pre_model.summary()  
layer = pre_model.get_layer('mixed10')
output = layer.output

In [22]:
x = MaxPooling2D()(output)
x = Flatten()(x)
x = Dense(512,activation = 'relu')(x)
x = Dropout(rate = 0.5)(x)
x = Dense(256,activation = 'relu')(x)
x = Dropout(rate = 0.5)(x)
x = Dense(128,activation='relu')(x)
x = Dropout(rate = 0.5)(x)
x = Dense(3,activation = 'softmax')(x)

model = Model(inputs = pre_model.input,outputs = x)

In [ ]:
model.compile(optimizer=RMSprop(learning_rate=0.0001),loss = 'categorical_crossentropy',metrics = ['acc'])
history = model.fit(train,epochs = 10,verbose =1,validation_data=val,steps_per_epoch=14630/256,validation_steps=1500/256)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['acc']
val_acc = history.history['val_acc']
epochs = range(1,len(acc)+1)

plt.plot(epochs , loss,'r--',label = "Training loss")
plt.plot(epochs,val_loss,'g--',label = "Validation loss")
plt.title("Training and Validation loss vs No of Epochs")
plt.legend()
plt.show()

plt.plot(epochs , acc,'bo-',label = "Training Accuracy")
plt.plot(epochs,val_acc,'go-',label = "Validation Accuracy")
plt.title("Training and Validation accuracy vs No of Epochs")
plt.legend()
plt.show()

In [ ]:
from google.colab import files
images = files.upload()

Saving _113249919_hi061718491.jpg to _113249919_hi061718491.jpg


In [ ]:
from keras.preprocessing import image
types = ['Cat','Dog','Wildlife']
for fn in images.keys():
  path = '/content/' + fn
  img = image.load_img(path,target_size=(256,256))
  x = image.img_to_array(img)
  x=x/255
  x = np.expand_dims(x,axis=0)
  prediction = model.predict(x)
  print("The image you uploaded is of a {} 😀".format(types[np.argmax(prediction)]))
  print("The uploaded image is : ")
  plt.imshow(img)